In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from repos_tools import read_chembl_screen_results, add_b3db_permeabilities
import attila_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preliminary SQLite operations

See the `notebook/2022-05-09-AD-drugs/target_{go.sql,protein_family}.sql` scripts, which produce `results/2022-05-09-AD-drugs/target-{go,pfam}.csv`, respectively.

In [3]:
pd.read_csv('../../results/2022-05-09-AD-drugs/target-go.csv')

,target_uniprot_ac,target_description,cs_organism,go_id,parent_go_id,pref_name,class_level,aspect,path
0,P02708,Acetylcholine receptor subunit alpha,Homo sapiens,GO:0005886,GO:0016020,plasma membrane,2,C,cellular_component membrane plasma membrane
1,P02708,Acetylcholine receptor subunit alpha,Homo sapiens,GO:0006810,GO:0008150,transport,1,P,biological_process transport
2,P02708,Acetylcholine receptor subunit alpha,Homo sapiens,GO:0007165,GO:0008150,signal transduction,1,P,biological_process signal transduction
3,P02708,Acetylcholine receptor subunit alpha,Homo sapiens,GO:0007267,GO:0008150,cell-cell signaling,1,P,biological_process cell-cell signaling
4,P02708,Acetylcholine receptor subunit alpha,Homo sapiens,GO:0016020,GO:0005575,membrane,1,C,cellular_component membrane
...,...,...,...,...,...,...,...,...,...
64852,P43630,Killer cell immunoglobulin-like receptor 3DL2,Homo sapiens,GO:0005515,GO:0003674,protein binding,1,F,molecular_function protein binding
64853,P43630,Killer cell immunoglobulin-like receptor 3DL2,Homo sapiens,GO:0005886,GO:0016020,plasma membrane,2,C,cellular_component membrane plasma membrane
64854,P43630,Killer cell immunoglobulin-like receptor 3DL2,Homo sapiens,GO:0006950,GO:0008150,response to stress,1,P,biological_process response to stress
64855,P43630,Killer cell immunoglobulin-like receptor 3DL2,Homo sapiens,GO:0016020,GO:0005575,membrane,1,C,cellular_component membrane


In [4]:
pd.read_csv('../../results/2022-05-09-AD-drugs/target-pfam.csv')

,target_uniprot_ac,target_description,cs_organism,protein_class_desc,pfam_level_1,pfam_level_2,pfam_level_3,pfam_level_4,pfam_level_5
0,P02708,Acetylcholine receptor subunit alpha,Homo sapiens,ion channel lgic ach chrn alpha,Ion channel,Ligand-gated ion channel,Nicotinic acetylcholine receptor,Nicotinic acetylcholine receptor alpha subunit,NaN
1,P04637,Cellular tumor antigen p53,Homo sapiens,transcription factor,Transcription factor,NaN,NaN,NaN,NaN
2,P05106,Integrin beta-3,Homo sapiens,membrane receptor,Membrane receptor,NaN,NaN,NaN,NaN
3,P05556,Integrin beta-1,Homo sapiens,membrane receptor,Membrane receptor,NaN,NaN,NaN,NaN
4,P06493,Cyclin-dependent kinase 1,Homo sapiens,enzyme kinase protein kinase cmgc cdk cdc2,Enzyme,Kinase,Protein Kinase,CMGC protein kinase group,CMGC protein kinase CDK family
...,...,...,...,...,...,...,...,...,...
4826,ENSG00000284190,microRNA 21,Homo sapiens,unclassified,Unclassified protein,NaN,NaN,NaN,NaN
4827,ENSG00000207827,microRNA 30a,Homo sapiens,unclassified,Unclassified protein,NaN,NaN,NaN,NaN
4828,ENSG00000090339,Intercellular adhesion molecule-1 mRNA 3'UTR,Homo sapiens,unclassified,Unclassified protein,NaN,NaN,NaN,NaN
4829,ENSG00000117601,Antithrombin mRNA,Homo sapiens,unclassified,Unclassified protein,NaN,NaN,NaN,NaN


In [5]:
%connect_info

{
  "shell_port": 50041,
  "iopub_port": 50042,
  "stdin_port": 50043,
  "control_port": 50045,
  "hb_port": 50044,
  "ip": "127.0.0.1",
  "key": "cbd427c6-5a5705eef9d59a43590105e2",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-09ba88b1-f007-452a-9b3d-0f9004091505.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
